<a href="https://colab.research.google.com/github/aleksandr329/SCAN_project/blob/Anna_branch/SCAN_%D0%BE%D1%82%D0%BA%D1%80%D1%8B%D1%82%D0%B8%D0%B5_%D1%84%D0%B0%D0%B9%D0%BB%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re

# Открытие датасета.

In [3]:
# для открытия и чтения файла с данным c Google.Drive через Google.Colab

from google.colab import drive
drive.mount('/content/drive')
path_DF = '/content/drive/MyDrive/Data_Science/Хакатон_SCAN/Queries.txt'

Mounted at /content/drive


In [4]:
# В файле Queries.txt приведено множество поисковых запросов клиентов продукта СКАН
# (формат: UserId Query - разделитель табуляция).

data = pd.read_csv(path_DF, sep='\t', header=None, names=['UserId', 'Query'])
data.head()

,UserId,Query
0,00000000-0000-0000-0000-000000000000,"((Text(""речные круизы"")) AND (Metadata(SourceC..."
1,00000000-0000-0000-0000-000000000000,"((Text(""Поезд"")) AND (Metadata(SourceCategory(..."
2,00000000-0000-0000-0000-000000000000,"((((Exact(Text(""Яндекс.Директ"")))) OR ((Text(""..."
3,00000000-0000-0000-0000-000000000000,"((Text(""отели"")) AND ((Metadata(SourceTopic({3..."
4,00000000-0000-0000-0000-000000000000,"((((Text(""российских""))) OR ((Location(Id(4C8B..."


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28371 entries, 0 to 28370
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   UserId  28371 non-null  object
 1   Query   28371 non-null  object
dtypes: object(2)
memory usage: 443.4+ KB


In [6]:
data.describe(include=object)

,UserId,Query
count,28371,28371
unique,1085,27593
top,00000000-0000-0000-0000-000000000000,"((Exact(Text(""9ff8d3d3-7624-490e-acd8-bfb2e3cf..."
freq,2219,7


In [7]:
print(f'Найдено дубликатов: {data.duplicated().sum()}')
print(f'Найдено пропусков в данных:\n {data.isna().sum()}')

Найдено дубликатов: 0
Найдено пропусков в данных:
 UserId    0
Query     0
dtype: int64


# Открытие json.

In [8]:
from google.colab import drive
drive.mount('/content/drive')
path_JSON = '/content/drive/MyDrive/Data_Science/Хакатон_SCAN/ThemesWordClouds.json'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import json
from pprint import pprint                                    # Импортируем функцию pprint() для печати журнала

with open(path_JSON) as f:                     # Открываем файл и связываем его с объектом "f"
  words_clouds_data = json.load(f)                           # Загружаем содержимое открытого файла в переменную words_clouds_data

# Открываем как датафрейм
#wc_data = pd.DataFrame(words_clouds_data)

pprint(words_clouds_data[15]['WordCloud'])   #Ключей 572 (от 0 до 571)

['займы',
 'кредиты',
 'кредит',
 'заем',
 'ссуда',
 'кредитный',
 'кредитование',
 'программа',
 'линия',
 'предложение',
 'пакет',
 'портфель',
 'договор',
 'соглашение',
 'фабрика',
 'сервис',
 'сумма',
 'объем',
 'получение',
 'кредита',
 'получить',
 'привлечь',
 'банки',
 'банк']


In [12]:
wc_data = pd.DataFrame(words_clouds_data)
wc_data.head()

,Theme,WordCloud
0,"Письма, указы, заявления от госорганов","[письмо, указ, поправка, заявление, сообщение,..."
1,Истец,"[истец, после, основании, иск, иск, суд, требо..."
2,Ответчик,"[против, ответчик, иск, действие, поведение, с..."
3,Преступник,"[преступник, уличить, уличать, разыскивать, об..."
4,Потерпевшая сторона,"[потерпевшая, сторона, покушение, нападение, о..."


In [16]:
wc_data.Theme.nunique()

572

# Прочее.

In [ ]:
# Фразовые запросы

phrase_queries = ['OrderedText', 'Text', 'OrderedWildcard', 'Wildcard',
                  'OrderedTextAnyDistance', 'InSameSentence', 'InSameParagraph',
                  'Exact']

# Запросы объектов и размерных величин
mesure_queries = ['Id', 'Metadata', 'Name', 'Org', 'Location', 'Trademark',
                  'IndustrialSector', 'IndustrialSectorPositiveTonality',
                  'IndustrialSectorNegativeTonality', 'NewsCluster', 'Date',
                  'Dim', 'AuthorName']

# Запросы фактов, рубрик и прямой речи
fact_queries = ['Speech', 'Event', 'Subject', 'EventType']

# Запросы источников
source_queries = ['Source', 'SourceLevel', 'SourceCategory', 'SourceRegion', 'SourceTopic']

# Остальное
others_queries = ['Language', 'DocumentType']

In [ ]:
'''from collections import Counter
>>> cnt = Counter(['red', 'blue', 'red', 'green', 'blue', 'blue'])
>>> dict(cnt)'''

"from collections import Counter\ncnt = Counter(['red', 'blue', 'red', 'green', 'blue', 'blue'])\ndict(cnt)"

In [ ]:
'''popular_query = []
for query in data.Query:
  query.lower()
  find = query.find("")'''

'popular_query = []\nfor query in data.Query:\n  query.lower()\n  find = query.find("")'

In [31]:

import re
#import Stemmer

#STEMMER = Stemmer.Stemmer('russian')

def tokenize(text):
    return text.split()

#def lowercase_filter(tokens):
    #return [token.lower() for token in tokens]

#def punctuation_filter(tokens):
    #return [PUNCTUATION.sub('', token) for token in tokens]

def stopword_filter(tokens):
    return [token for token in tokens if token not in STOPWORDS]

def stem_filter(tokens):
    return STEMMER.stemWords(tokens)

def analyze(text):
    tokens = tokenize(text)
    #tokens = lowercase_filter(tokens)
    #tokens = punctuation_filter(tokens)
    #tokens = stopword_filter(tokens)
    tokens = stem_filter(tokens)

    return [token for token in tokens if token]

# Извлечение текста запроса (в работе).

In [138]:
df = data.copy()

In [139]:
# удаляем скобки
df['Query'] = df['Query'].apply(lambda query: query.replace('(', ' '))
df['Query'] = df['Query'].apply(lambda query: query.replace(')', ' '))

In [140]:
# выделение текста запроса через регулярку
# не организации, не персоны, не локации = ['Org', 'Name', 'Location']

def get_txt(query):
  if 'Org' or 'Name' or 'Location' not in query:  # не организации, не персоны, не локации = ['Org', 'Name', 'Location']
    query.split(' ')
    pattern = r'["]\S+\s*\S+\s*\S+["]'
    result = re.findall(pattern, query)
    return result
  else:
    return None


df['Text'] = df['Query'].apply(lambda query: get_txt(query)).astype(str)
#df['Text'] = df['Text'].apply(lambda x: str(x))

In [141]:
df['Text'] = df['Text'].apply(lambda x: x.split(','))
df = df.explode('Text', ignore_index=True)
df['Text'] = df['Text'].astype(str)

In [142]:
def edit_text(text):
  for letter in text:
    if letter =='[':
      return text.replace('[','').strip()
    if letter == ']':
      return text.replace(']','').strip()
    else:
      return text.strip().strip()

In [143]:
df['Text'] = df['Text'].apply(lambda text: edit_text(text))

In [144]:
df.head(20)

,UserId,Query,Text
0,00000000-0000-0000-0000-000000000000,"Text ""речные круизы"" AND Metadata SourceC...","'""речные круизы""']"
1,00000000-0000-0000-0000-000000000000,"Text ""Поезд"" AND Metadata SourceCategory ...","'""Поезд""']"
2,00000000-0000-0000-0000-000000000000,"Exact Text ""Яндекс.Директ"" OR Text ""...","'""Яндекс.Директ""'"
3,00000000-0000-0000-0000-000000000000,"Exact Text ""Яндекс.Директ"" OR Text ""...","'""на наш Telegram""'"
4,00000000-0000-0000-0000-000000000000,"Exact Text ""Яндекс.Директ"" OR Text ""...","'""Яндекс.Дзен""'"
5,00000000-0000-0000-0000-000000000000,"Exact Text ""Яндекс.Директ"" OR Text ""...","'""профиль в Инстаграм""'"
6,00000000-0000-0000-0000-000000000000,"Exact Text ""Яндекс.Директ"" OR Text ""...","'""Telegram-канал""']"
7,00000000-0000-0000-0000-000000000000,"Text ""отели"" AND Metadata SourceTopic {3...","'""отели""']"
8,00000000-0000-0000-0000-000000000000,"Text ""российских"" OR Location Id 4C8B...","'""российских""']"
9,00000000-0000-0000-0000-000000000000,"Text ""Москва"" AND Metadata SourceCategory...","'""Москва""']"
